# BoxOfficeMojo Data Cleaner

In [1]:
%pylab inline
#% config InlineBackend.figure_format = 'svg'

import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


## Load the dataframe from its pickle file and perform basic datatype transformations/cleaning

### Define which years to clean

In [ ]:
years = [2020]
#years = range(2019,1990,-1)

### Run loop to clean for every year

In [5]:
for year in years:
    filepath = ('./data/mojo_'+str(year)+'_movies.pkl')
    df = pd.read_pickle(filepath)

    # Cleaning
    df = df.drop(columns='index')
    df.fillna(value=np.nan, inplace=True)
    # save to pickle
    df.to_pickle(path=('./data/mojo_'+str(year)+'_movies_cleaned.pkl'))

### Check the last dataframe

In [ ]:
df.head()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Movie_Title           242 non-null    object        
 1   Domestic_Distributor  237 non-null    object        
 2   Domestic_Total_Gross  242 non-null    int64         
 3   Runtime               228 non-null    float64       
 4   Rating                119 non-null    object        
 5   Release_Date          242 non-null    datetime64[ns]
 6   Budget                49 non-null     float64       
 7   Cast1                 235 non-null    object        
 8   Cast2                 232 non-null    object        
 9   Cast3                 228 non-null    object        
 10  Cast4                 224 non-null    object        
 11  Director              235 non-null    object        
 12  Writer                223 non-null    object        
 13  Producer            